This code includes fine-tuning of the phi-4 model, followed by testing on a separate dataset. It also contains a comparison between the fine-tuned model and the RAG-enhanced model.

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.3 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### Testing the model for test dataset

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/phi-4",
    device_map="auto",
    torch_dtype=torch.float16  # veya float32 Colab durumuna göre
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/phi-4")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

model-00006-of-00006.safetensors:   0%|          | 0.00/4.62G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

OSError: unsloth/phi-4 does not appear to have files named ('model-00001-of-00006.safetensors', 'model-00002-of-00006.safetensors', 'model-00003-of-00006.safetensors', 'model-00004-of-00006.safetensors', 'model-00005-of-00006.safetensors', 'model-00006-of-00006.safetensors'). Checkout 'https://huggingface.co/unsloth/phi-4/tree/main'for available files.

In [ ]:
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/fine_tuned_phi4_2")
model = model.merge_and_unload()  # Eğer LoRA'yı ana modelle birleştirmek istersen


In [ ]:
import pandas as pd
# 2. Excel dosyasını oku (dosya adını güncelle)
excel_path = "/content/Test_dataset.xlsx"
df = pd.read_excel(excel_path)

# 3. Soruları al
questions = df["Question"].dropna().tolist()

In [ ]:
# 4. Cevapları üret
def generate_answer(question, model, tokenizer, max_length=100):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 5. Sonuçları topla
results = []
for q in questions:
    answer = generate_answer(q, model, tokenizer)
    results.append({"Question": q, "Answer": answer})
    print(f"\nQ: {q}\nA: {answer}")



NameError: name 'model' is not defined

In [ ]:
# 6. Sonuçları kaydet (istersen)
results_df = pd.DataFrame(results)
results_df.to_excel("/content/test_answers.xlsx", index=False)

<a name="Data"></a>
### Data Prep
We now use the `Phi-4` format for conversation style finetunes. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Phi-4 renders multi turn conversations like below:

```
<|im_start|>user<|im_sep|>Hello!<|im_end|>
<|im_start|>assistant<|im_sep|>Hi! How can I help?<|im_end|>
<|im_start|>user<|im_sep|>What is 2+2?<|im_end|>
```


In [4]:
import pandas as pd


df = pd.read_excel("/content/datasets.xlsx")


In [5]:
import pandas as pd
import json


# Her soru-cevap çiftini formatla ve JSONL satırı olarak hazırla
lines = []
for _, row in df.iterrows():
    question = str(row['Question']).strip()
    answer = str(row['Answer']).strip()
    chat_format = f"<|im_start|>user<|im_sep|>{question}<|im_end|><|im_start|>assistant<|im_sep|>{answer}<|im_end|>"
    lines.append({"text": chat_format})

# JSONL formatında kaydetmek istersen:
with open("phi4_dataset.jsonl", "w", encoding="utf-8") as f:
    for line in lines:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

# Değişken olarak da kullanabilirsin
dataset = lines


In [6]:
dataset

[{'text': '<|im_start|>user<|im_sep|>What is the main purpose of Power Apps within the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>To allow users to build low-code/no-code custom business applications.<|im_end|>'},
 {'text': '<|im_start|>user<|im_sep|>How do Power Apps and Power Automate complement each other in the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>Power Apps is used to build apps, while Power Automate enables automation of workflows within those apps.<|im_end|>'},
 {'text': "<|im_start|>user<|im_sep|>What does 'within app context' mean in Power Automate licensing?<|im_end|><|im_start|>assistant<|im_sep|>It means flows can only access data and services directly related to the app that triggered them.<|im_end|>"},
 {'text': '<|im_start|>user<|im_sep|>What is the difference between attended and unattended RPA in Power Automate?<|im_end|><|im_start|>assistant<|im_sep|>Attended RPA runs with user interaction, while unattended RPA runs autonomously.<|im_en

In [7]:
dataset[0]["text"]

'<|im_start|>user<|im_sep|>What is the main purpose of Power Apps within the Power Platform?<|im_end|><|im_start|>assistant<|im_sep|>To allow users to build low-code/no-code custom business applications.<|im_end|>'

In [8]:
from datasets import Dataset

# JSONL olarak oluşturduğun listeyi HuggingFace Dataset'e dönüştür
hf_dataset = Dataset.from_list(dataset)


<a name="Train"></a>
### Train the model


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/98 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [10]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=2):   0%|          | 0/98 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


We verify masking is actually done:

In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>user<|im_sep|>Why would an organization use Power Apps per app license instead of Premium?<|im_end|><|im_start|>assistant<|im_sep|>For limited use cases where only one or two apps need to be run per user.<|im_end|>'

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                     For limited use cases where only one or two apps need to be run per user.<|im_end|>'

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 98 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 65,536,000/4,000,000,000 (1.64% trained)


Step,Training Loss
1,5.832700
2,5.844000
3,5.061700
4,4.808300
5,4.602300
6,3.975900
7,3.100900
8,2.570400
9,2.808200
10,2.864000


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

We use `min_p = 0.1` and `temperature = 1.5`.

In [14]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
import torch
import re

tokenizer = get_chat_template(tokenizer, chat_template = "phi-4")
FastLanguageModel.for_inference(model)

def parse_phi_chat(text):
    pattern = r"<\|im_start\|>(user|assistant)<\|im_sep\|>(.*?)<\|im_end\|>"
    matches = re.findall(pattern, text, re.DOTALL)
    return [{"role": role.strip(), "content": content.strip()} for role, content in matches]

text = dataset[0]["text"]
messages = parse_phi_chat(text)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda" if torch.cuda.is_available() else "cpu")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 100,
    use_cache = True,
    temperature = 1.0,
    top_p = 0.95,
)

#Sadece üretilen kısmı decode et
generated = outputs[0][inputs.shape[-1]:]
print(tokenizer.decode(generated, skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Power Automate is used for automating workflows and business processes.


 You can also use a `TextStreamer` for continuous inference - so you can see The generation token by token, instead of waiting the whole time!

## 🧪 Inference on Custom Dataset and Exporting Model Outputs to CSV

This section performs **inference using a fine-tuned language model** (e.g., Phi-4 with LoRA adapters) on a custom dataset containing chat-style prompts and expected responses. The model's outputs are collected and saved into a CSV file for further analysis.

### 📝 Dataset Format
Each entry in the dataset is a single string with special formatting using tokens:

<|im_start|>user<|im_sep|>QUESTION<|im_end|>

<|im_start|>assistant<|im_sep|>EXPECTED_ANSWER<|im_end|>



### 🎯 Goal
- Extract the `QUESTION` part from each dataset item.
- Generate a response using the model.
- Compare it with the `EXPECTED_ANSWER` (optional).
- Save all results into a `model_outputs.csv` file.

### 📤 Output CSV Columns
- **question**: User input extracted from the dataset.
- **expected_answer**: The reference assistant answer.
- **generated_answer**: The model’s generated response.

This step is essential for validating how well the model performs on custom examples and helps in error analysis or evaluation.


In [15]:
import pandas as pd
import torch
from openpyxl import Workbook

# 1. Test verisini oku
test_df = pd.read_excel("/content/Test_dataset.xlsx")  # yeni test dosyası

results = []

for i, question in enumerate(test_df['Question']):
    question = str(question).strip()
    messages = [{"role": "user", "content": question}]

    # 2. Tokenize et ve tensore dönüştür
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    # 3. Modelden cevap üret
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            use_cache=True,
        )

    # 4. Decode et
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # assistant cevabını çek
    if "<|im_start|>assistant<|im_sep|>" in decoded_output:
        generated_answer = decoded_output.split("<|im_start|>assistant<|im_sep|>")[-1].split("<|im_end|>")[0].strip()
    else:
        generated_answer = decoded_output.strip()

    results.append((question, generated_answer))

# 5. Sonuçları Excel dosyasına yaz
wb = Workbook()
ws = wb.active
ws.title = "Test Results"

ws.append(["Question", "Generated Answer"])

for row in results:
    ws.append(row)

output_path = "new_test_model_outputs.xlsx"
wb.save(output_path)
print(f"✅ Test sonuçları '{output_path}' dosyasına kaydedildi.")


✅ Test sonuçları 'new_test_model_outputs.xlsx' dosyasına kaydedildi.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How can users customize Copilot behavior?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

By using specific prompts and commands<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

In [ ]:
from google.colab import drive
import shutil
import os

# 1. Google Drive'ı mount et
drive.mount('/content/drive')

# 2. Kaynak klasör (modelin bulunduğu yer)
source_folder = "/content/lora_model"  # lora_model klasörün buradaysa
drive_folder = "/content/drive/MyDrive/fine_tuned_phi4"
os.makedirs(drive_folder, exist_ok=True)

# 3. Kopyalanacak dosyaların listesi
files_to_copy = [
    "adapter_model.safetensors",
    "adapter_config.json",
    "tokenizer.json",
    "tokenizer_config.json",
    "vocab.json",
    "merges.txt",
    "special_tokens_map.json"
]

# 4. Dosyaları kopyala
for file_name in files_to_copy:
    source_path = os.path.join(source_folder, file_name)
    if os.path.exists(source_path):
        shutil.copy(source_path, drive_folder)
        print(f"✅ {file_name} Drive'a kopyalandı.")
    else:
        print(f"⚠️ {file_name} bulunamadı, atlandı.")

print("📁 Tüm işlemler tamamlandı.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ adapter_model.safetensors Drive'a kopyalandı.
✅ adapter_config.json Drive'a kopyalandı.
✅ tokenizer.json Drive'a kopyalandı.
✅ tokenizer_config.json Drive'a kopyalandı.
✅ vocab.json Drive'a kopyalandı.
✅ merges.txt Drive'a kopyalandı.
✅ special_tokens_map.json Drive'a kopyalandı.
📁 Tüm işlemler tamamlandı.


# RAG

In [16]:
pip install PyMuPDF faiss-cpu sentence-transformers pandas openpyxl transformers unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# 1. PDF'ten metin çıkar
doc = fitz.open("dokuman.pdf")
text = "\n".join([page.get_text() for page in doc])

# 2. Metni parçalara ayır
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)
chunks = splitter.split_text(text)

# 3. Embed oluştur
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True)

# 4. FAISS index oluştur
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

# 5. RAG fonksiyonu
def retrieve_context(question, k=3):
    q_emb = embedder.encode([question])
    distances, indices = faiss_index.search(q_emb, k)
    return [chunks[i] for i in indices[0]]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Prompt+Model

In [18]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/phi-4",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model.eval()


==((====))==  Unsloth 2025.5.3: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [19]:
df = pd.read_excel("Test_dataset.xlsx")  # test veri seti

rag_results = []

for i, row in df.iterrows():
    question = row["Question"]
    context = "\n".join(retrieve_context(question))

    # Phi-4 mesaj formatı
    messages = [{"role": "user", "content": f"{context}\nSoru: {question}"}]

    # Prompt'u modelin formatına göre hazırla
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Yanıtı ayıkla
    try:
        answer = decoded_output.split("<|im_start|>assistant<|im_sep|>")[-1].split("<|im_end|>")[0].strip()
    except IndexError:
        answer = decoded_output.strip()

    rag_results.append((question, context, answer))
    print(f"✅ Soru {i+1} işlendi.")


✅ Soru 1 işlendi.
✅ Soru 2 işlendi.
✅ Soru 3 işlendi.
✅ Soru 4 işlendi.
✅ Soru 5 işlendi.
✅ Soru 6 işlendi.
✅ Soru 7 işlendi.
✅ Soru 8 işlendi.
✅ Soru 9 işlendi.
✅ Soru 10 işlendi.
✅ Soru 11 işlendi.
✅ Soru 12 işlendi.
✅ Soru 13 işlendi.
✅ Soru 14 işlendi.
✅ Soru 15 işlendi.
✅ Soru 16 işlendi.
✅ Soru 17 işlendi.
✅ Soru 18 işlendi.
✅ Soru 19 işlendi.
✅ Soru 20 işlendi.


#### Save the results

In [20]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active
ws.title = "RAG Cevapları"
ws.append(["Question", "Context", "Generated Answer (RAG)"])

for row in rag_results:
    ws.append(row)

wb.save("test_rag_outputs.xlsx")
print("✅ Excel dosyası kaydedildi.")

✅ Excel dosyası kaydedildi.


In [21]:
print(f"{len(rag_results)} cevap üretildi.")

20 cevap üretildi.
